# This notebook calculates autocorrelation of dipole moments(C-CH3) from a molecular dynamics simulation trajecotry. This autocorrelation cab be used for estimating the NMR S2 order parameters 

In [ ]:
# loads trajecotry and topological information
%reset
import h5py
import numpy as np
import scipy.io as sio

# for doing the same analysis just change the names of the input and output files

# if trajectory and topology are saved with matlab code 
trajAndTopol = h5py.File("trajAndTopol.mat")

# if trajectory and topology are saved with pyhton code
trajAndTopol = np.load("trajAndTopol.npz")

# check the axes of the traj
traj.shape

# it should be like (frms, atms, coors)
# change the axes if it's necessary 
# traj = np.swapaxes(traj,...)

traj = trajAndTopol['traj']
resId = trajAndTopol['resId']
resNm = trajAndTopol['resNm']
atmNm = trajAndTopol['atmNm']
atmId = trajAndTopol['atmId']

In [ ]:
# prepares arrays to save the outputs
import numpy as np

# the residue numbers we want to calculate S2 order parameter for them
methylRes = [6,9,11,12,18,20,22,23,26,28,30,35,37,39,40,41,45,46,52,\
             58,59,61,64,66,69,70,74,75,77,78,81,84,85,87,88,89]

# for allocating an array to save methyl dipoles
numOfMethyl = 0

# keeps the residue Id for methyl groups
methylGrpRes = list()
methylResName = ()
methylResNames = list()

for res in methylRes:
    
    methylResName = resNm[np.where(resId==res)][0]
    
    # keep the name of the res that have methyl
    methylResNames.append(methylResName)
    
    if methylResName == 'ALA' or methylResName == 'THR':
        # has one methyl group
        numOfMethyl +=1
        methylGrpRes.append(res)
    else:
        # the rest have 2 methyl groups
        numOfMethyl +=2
        methylGrpRes.append(res)
        methylGrpRes.append(res)
        
        del methylResName 
del res 

methylResNames = np.array(methylResNames)
methylGrpRes = np.array(methylGrpRes)

In [ ]:
# this block calculatea and saves dipole moments for residues in all frames

import numpy as np
import scipy.io as sio

# for calculating the C-CH3 dipoles and saving them for each frame


numOfFrames = traj.shape[0]
# array that contains dipoles in all frames for all methyl groups (nFrm,numOfmeth,coor)
methylDipols = np.zeros([traj.shape[0],numOfMethyl,3])

# the array keep the resId, resNm and atoms names for dipoles
methylResIdResNmDipolAtms = np.empty((numOfMethyl,3),dtype=object)
methylResIdResNmDipolAtms[:,0] = methylGrpRes

# res for resdiue number and resN for residue Name
for res, resN in zip(methylRes,methylResNames):
    print res
    
    if resN == 'ILE':
        # for ILE  (CB-CG2, CG1-CD)
        ileAtoms = atmNm[np.where(resId==res)]
        ileAtomsId = atmId[np.where(resId==res)]

        CG2_res_ind = atmId[np.where(ileAtoms=='CG2')][0]-1
        CB_res_ind = atmId[np.where(ileAtoms=='CB')][0]-1

        CG1_res_ind = atmId[np.where(ileAtoms=='CG1')][0]-1
        CD_res_ind = atmId[np.where(ileAtoms=='CD')][0]-1

        CG2_ind = ileAtomsId[CG2_res_ind]-1
        CB_ind = ileAtomsId[CB_res_ind]-1

        CG1_ind = ileAtomsId[CG1_res_ind]-1
        CD_ind = ileAtomsId[CD_res_ind]-1
        
        # filling the second column with the name of the residue
        methylResIdResNmDipolAtms[np.where(methylGrpRes==res),1] = 'ILE'
        
        # finding the write place for saving dipoles of methyl groups 
        methyl1_ind = np.where(methylGrpRes==res)[0][0]
        methyl2_ind = np.where(methylGrpRes==res)[0][1]
        
        # filling the 3rd col with methyl atoms
        methylResIdResNmDipolAtms[methyl1_ind,2] = 'CG2_CB'
        methylResIdResNmDipolAtms[methyl2_ind,2] = 'CG1_CD'
        
        # Calculating dipole moments
        methylDipols[:,methyl1_ind,:] = traj[:,CG2_ind,:] - traj[:,CB_ind,:]
        methylDipols[:,methyl2_ind,:] = traj[:,CG1_ind,:] - traj[:,CD_ind,:]
        
        del (ileAtoms, ileAtomsId, CG2_res_ind, CB_res_ind, CG1_res_ind, CD_res_ind,
        CG2_ind, CB_ind, CG1_ind, CD_ind, methyl1_ind, methyl2_ind)


    elif resN == 'VAL':
        # for VAL (CB-CG1, CB-CG2)
        valAtoms = atmNm[np.where(resId==res)]
        valAtomsId = atmId[np.where(resId==res)]

        CG1_res_ind = atmId[np.where(valAtoms=='CG1')][0]-1
        CB_res_ind = atmId[np.where(valAtoms=='CB')][0]-1

        CG2_res_ind = atmId[np.where(valAtoms=='CG2')][0]-1
        
        CG1_ind = valAtomsId[CG1_res_ind]-1
        CB_ind = valAtomsId[CB_res_ind]-1

        CG2_ind = valAtomsId[CG2_res_ind]-1
        
        # filling the second column with the name of the residue
        methylResIdResNmDipolAtms[np.where(methylGrpRes==res),1] = 'VAL'
        
        # finding the write place for saving dipoles of methyl groups 
        methyl1_ind = np.where(methylGrpRes==res)[0][0]
        methyl2_ind = np.where(methylGrpRes==res)[0][1]
        
        # filling the 3rd col with methyl atoms
        methylResIdResNmDipolAtms[methyl1_ind,2] = 'CG1_CB'
        methylResIdResNmDipolAtms[methyl2_ind,2] = 'CG2_CB'
        
        # Calculating dipole moments
        
        methylDipols[:,methyl1_ind,:] = traj[:,CG1_ind,:] - traj[:,CB_ind,:]
        methylDipols[:,methyl2_ind,:] = traj[:,CG2_ind,:] - traj[:,CB_ind,:]
        

        del (valAtoms, valAtomsId, CG1_res_ind, CB_res_ind, CG2_res_ind,CG1_ind,
             CB_ind, CG2_ind, methyl1_ind, methyl2_ind)

    elif resN == 'LEU':
        # for LEU (CG-CD1, CG-CD2)
        leuAtoms = atmNm[np.where(resId==res)]
        leuAtomsId = atmId[np.where(resId==res)]

        CD1_res_ind = atmId[np.where(leuAtoms=='CD1')][0]-1
        CG_res_ind = atmId[np.where(leuAtoms=='CG')][0]-1

        CD2_res_ind = atmId[np.where(leuAtoms=='CD2')][0]-1
        
        CD1_ind = leuAtomsId[CD1_res_ind]-1
        CG_ind = leuAtomsId[CG_res_ind]-1

        CD2_ind = leuAtomsId[CD2_res_ind]-1
        
        # filling the second column with the name of the residue
        methylResIdResNmDipolAtms[np.where(methylGrpRes==res),1] = 'LEU'
        
        # finding the write place for saving dipoles of methyl groups 
        methyl1_ind = np.where(methylGrpRes==res)[0][0]
        methyl2_ind = np.where(methylGrpRes==res)[0][1]
        
        # filling the 3rd col with methyl atoms
        methylResIdResNmDipolAtms[methyl1_ind,2] = 'CD1_CG'
        methylResIdResNmDipolAtms[methyl2_ind,2] = 'CD2_CG'
        
        # Calculating dipole moments
        methylDipols[:,methyl1_ind,:] = traj[:,CD1_ind,:] - traj[:,CG_ind,:]
        methylDipols[:,methyl2_ind,:] = traj[:,CD2_ind,:] - traj[:,CG_ind,:]
        

        del (leuAtoms, leuAtomsId, CD1_res_ind, CG_res_ind, CD2_res_ind, CD1_ind, CD2_ind,
             CG_ind, methyl1_ind, methyl2_ind)

    elif resN == 'THR':
        # for THR (CB-CG2)
        thrAtoms = atmNm[np.where(resId==res)]
        thrAtomsId = atmId[np.where(resId==res)]

        CG2_res_ind = atmId[np.where(thrAtoms=='CG2')][0]-1
        CB_res_ind = atmId[np.where(thrAtoms=='CB')][0]-1

        CG2_ind = thrAtomsId[CG2_res_ind]-1
        CB_ind = thrAtomsId[CB_res_ind]-1
        
        # filling the second column with the name of the residue
        methylResIdResNmDipolAtms[np.where(methylGrpRes==res),1] = 'THR'
        
        # finding the write place for saving dipoles of methyl groups 
        methyl_ind = [np.where(methylGrpRes==res)][0][0][0]
        
        
        # filling the 3rd col with methyl atoms
        methylResIdResNmDipolAtms[methyl_ind,2] = 'CG2_CB'
        
        # Calculating dipole moments
        methylDipols[:,methyl_ind,:] = traj[:,CG2_ind,:] - traj[:,CB_ind,:]
                

        del thrAtoms, thrAtomsId, CG2_res_ind, CB_res_ind, CG2_ind, CB_ind, methyl_ind

    else:
        # for ALA (CA-CB)
        alaAtoms = atmNm[np.where(resId==res)]
        alaAtomsId = atmId[np.where(resId==res)]

        CB_res_ind = atmId[np.where(alaAtoms=='CB')][0]-1
        CA_res_ind = atmId[np.where(alaAtoms=='CA')][0]-1

        CB_ind = alaAtomsId[CB_res_ind]-1
        CA_ind = alaAtomsId[CA_res_ind]-1
        
        # filling the second column with the name of the residue
        methylResIdResNmDipolAtms[np.where(methylGrpRes==res),1] = 'ALA'
        
        # finding the write place for saving dipoles of methyl groups 
        methyl_ind = [np.where(methylGrpRes==res)][0][0][0]
        
        
        # filling the 3rd col with methyl atoms
        methylResIdResNmDipolAtms[methyl_ind,2] = 'CB_CA'
        
        # Calculating dipole moments
        methylDipols[:,methyl_ind,:] = traj[:,CB_ind,:] - traj[:,CA_ind,:]
                

        del alaAtoms, alaAtomsId, CB_res_ind, CA_res_ind, CB_ind, CA_ind, methyl_ind
        
#del  atmId, atmNm, methylGrpRes, methylRes, methylResNames, numOfFrames, numOfMethyl, traj, res, resId, resN, resNm

# saving the array that contains C-CH3 dipoles in all frames and another array that contains the inforamtion of 
# residue number, name and atoms that compose the dipole
sio.savemat('dipoles.mat',{'methylDipols':methylDipols, 'methylResIdResNmDipolAtms':methylResIdResNmDipolAtms})

In [ ]:
# Calculates autocorrelation 

%reset

import numpy as np
import scipy.io as sio
from numba import jit

data = sio.loadmat('dipoles.mat')

methylDipols = data['methylDipols']
methylResIdResNmDipolAtms = data['methylResIdResNmDipolAtms']


# maximum time delay 10ns or 40000*0.25ps 
maxDelT = 9


# to calculate the unit vector of dipoles
dipLen = np.sqrt(np.sum(np.square(methylDipols),axis=2))
methDipUnit = methylDipols/dipLen[:,:,np.newaxis]

# to have coordinates as the second dimension
methDipUnit = np.swapaxes(methDipUnit, 2, 1)

del data, methylDipols, dipLen

# calculating the dipole correlations

# number of frames
nFrm = methDipUnit.shape[0]

# C saves the correlations
C = np.zeros([methDipUnit.shape[2], maxDelT+1])

# saves the number of points were used to calculate autocorrelation 
nDataPnts = np.zeros([1, maxDelT+1])

@jit # for improvre the speed of calculation
def makeInd(delT, nFrm, nDataPnts):
    ind = np.zeros(nFrm-delT, dtype=int)
    nDataPnts[0, delT] = len(ind)
    
    # the indeces for t0s
    temInd = np.arange(1,nFrm+1,delT)[:-1]
    temIndLenOld = len(temInd)
    ind[0:temIndLenOld] = temInd

    if delT > 1:
        for i in xrange(1,delT):
            temInd = np.arange(1+i,nFrm+1,delT)[:-1]
            temIndLenNew = len(temInd)
            ind[temIndLenOld:temIndLenOld + temIndLenNew] = temInd
            temIndLenOld = temIndLenOld + temIndLenNew


    #print delT
    return ind

@jit
def autoCorCal(ind, methDipUnit, C):
    t0 = ind-1
    aveDotDip = np.sum(np.multiply(methDipUnit[t0,:,:], methDipUnit[t0+delT,:,:]),axis=1)

    # formula for calculating correlations
    p2 = 0.5*(3*np.square(aveDotDip) - 1)
    # correlations
    C[:,delT] = np.mean(p2,axis=0)
    # to save the number of data points for calculating the uncertanities

    #del t0, aveDotDip, p2
    #print delT


for delT in xrange(1,maxDelT+1):
    ind = makeInd(delT, nFrm, nDataPnts)
    autoCorCal(ind, methDipUnit, C)
    print delT
C[:,0] = 1
sio.savemat('Cor.mat',{'Corr':C, 'nDataPnts':nDataPnts, 'methylResIdResNmDipolAtms':methylResIdResNmDipolAtms})

In [ ]:
# Calculates autocorrelation 
# non-vectorized version
%reset

import numpy as np
import scipy.io as sio
from numba import jit

data = sio.loadmat('pdz2ApoDipoles.mat')

methylDipols = data['methylDipols']
methylResIdResNmDipolAtms = data['methylResIdResNmDipolAtms']


# maximum time delay 10ns or 40000*0.25ps 
maxDelT = 9


# to calculate the unit vector of dipoles
dipLen = np.sqrt(np.sum(np.square(methylDipols),axis=2))
methDipUnit = methylDipols/dipLen[:,:,np.newaxis]

# to have coordinates as the second dimension
methDipUnit = np.swapaxes(methDipUnit, 2, 1)

del data, methylDipols, dipLen

# calculating the dipole correlations

# number of frames
nFrm = methDipUnit.shape[0]

# C saves the correlations
C = np.zeros([methDipUnit.shape[2], maxDelT+1])

# saves the number of points were used to calculate autocorrelation 
nDataPnts = np.zeros([1, maxDelT+1])
       

@jit # for improvre the speed of calculation
def autoCorCal(maxDelT, methDipUnit, me, C, nDataPnts):
    nFrm = methDipUnit.shape[0]

    ctr1 = 0
    p2_1 = 0
        
    ctr2 = 0
    p2_2 = 0
            
    for delT in xrange(1,maxDelT+1):
        ctr1 = 0
        p2_1 = 0
        
        
        for strFrm in xrange(1,1+delT):
            
            if strFrm+delT == nFrm:
                aveDotDip2 = np.dot(methDipUnit[strFrm-1,:,me-1],methDipUnit[strFrm-1+delT,:,me-1])
                p2_2 = p2_2 + 0.5*(3*aveDotDip2**2-1)
                ctr2 = ctr2+1
            else:
                
                while strFrm+delT <= nFrm:
                    aveDotDip2 = np.dot(methDipUnit[strFrm-1,:,me-1],methDipUnit[strFrm-1+delT,:,me-1])
                    p2_2 = p2_2 + 0.5*(3*aveDotDip2**2-1)
                    ctr2 = ctr2+1
                    
                    strFrm=strFrm+delT 
                ctr1 = ctr1+ctr2
                
                p2_1 = p2_1+p2_2
                ctr2=0
                p2_2 = 0
            
        p2_1 = p2_1/float(ctr1)
        
        C[me-1,delT] = p2_1
        nDataPnts[0, delT] = ctr1


    
for me in range(1,methDipUnit.shape[-1]+1):
    print('me',me)
    autoCorCal(maxDelT, methDipUnit, me, C, nDataPnts)
    
C[:,0] = 1
sio.savemat('CorNonVectorized.mat',{'Corr':C, 'nDataPnts':nDataPnts, 'methylResIdResNmDipolAtms':methylResIdResNmDipolAtms})